In [2]:
!pip install numpy pandas matplotlib seaborn scipy yfinance

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.optimize import minimize
from scipy.stats import norm
import warnings
warnings.filterwarnings('ignore')

#display settings
plt.style.use('seaborn-v0_8-darkgrid')
pd.set_option('display.max_columns', None)

# Introduction & Project Overview

# Part 1: Liability Modelling
## Setting Up Member Demographics and Assumptions

In [10]:
#Actuarial Assumptions
assumptions = {
    'discount_rate': 0.06,
    'salary_growth': 0.03,
    'retirement_age': 65,
    'mortality_improvement': 0.06,
    'benifit_accrual': 0.02
}

#Display Assumptions
print("Pension Fund Assumptions:")
print("-" * 40)
for key, value in assumptions.items():
    print(f"{key.replace('_', ' ').title()}: {value}")

Pension Fund Assumptions:
----------------------------------------
Discount Rate: 0.06
Salary Growth: 0.03
Retirement Age: 65
Mortality Improvement: 0.06
Benifit Accrual: 0.02


In [11]:
# Create a sample pension plan membership
np.random.seed(42)  # For reproducibility
n_members = 1000  # Number of plan members
ages = np.random.randint(25,65,n_members)
max_possible_service = ages -18
years_of_service = [np.random.randint(0, min(max_years, 35) + 1) for max_years in max_possible_service]


members = pd.DataFrame({
    'member_id': range(1, n_members + 1),
    'current_age': ages,
    'current_salary': np.random.uniform(40000, 150000, n_members),
    'years_of_service': years_of_service
})

# Display first few members
print("\nSample Plan Members:")
members.head(10)


Sample Plan Members:


,member_id,current_age,current_salary,years_of_service
0,1,63,58481.562852,13
1,2,53,121056.111944,21
2,3,39,134907.678530,10
3,4,32,63841.378010,6
4,5,45,50528.601107,0
5,6,63,42600.244465,20
6,7,43,110616.865049,25
7,8,47,106780.343970,13
8,9,35,100136.715399,3
9,10,35,65514.180573,0
